In [17]:
import numpy as np # работа с массивами и сруктурой наборов данных
import os # работа с путями файлов
import tensorflow as tf

from keras.callbacks import TensorBoard
from sklearn.model_selection import train_test_split # разделение данных для тестирования и тренировки
from keras.utils import to_categorical

In [18]:
DATA_PATH = './np_sequence_data'
model_path = './models/dinamic_model.h5'

sequence_length = 30

actions = ['none', 'down', 'right', 'shake', 'static', 'to_fist', 'up', 'z_write']

In [19]:
# создание словаря со значениями из actions
label_map = {label:num for num, label in enumerate(actions)}
label_map

{'none': 0,
 'down': 1,
 'right': 2,
 'shake': 3,
 'static': 4,
 'to_fist': 5,
 'up': 6,
 'z_write': 7}

In [20]:
sequences, labels = [], [] # последовательности и метки
for action in actions:
    # считаем количество папок с массивами ключевых точек
    with os.scandir(os.path.join(DATA_PATH, action)) as entries:
        no_sequences = sum(entry.is_dir() for entry in entries)

    for sequence in range(no_sequences):
        window = []
    
        for frame_num in range(sequence_length):
            res = np.load(os.path.join(DATA_PATH, action, str(sequence), "{}.npy".format(frame_num)))
            window.append(res)
        sequences.append(window)
        labels.append(label_map[action])

In [27]:
print(np.array(sequences).shape)
print(np.array(labels).shape)

(800, 30, 132)
(800,)


In [28]:
# представление значений из sequences, labels в удобном виде

X = np.array(sequences)
y = to_categorical(labels).astype(int)

In [29]:
# разделение данных на тестовые и тренировочные
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.10, random_state=99)

In [8]:
NUM_CLASSES = 8

model = tf.keras.models.Sequential([
    tf.keras.layers.LSTM(128, return_sequences=True, activation='relu', input_shape=(30,132)),
    tf.keras.layers.LSTM(256, return_sequences=True, activation='relu'),
    tf.keras.layers.LSTM(128, return_sequences=False, activation='relu'),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(NUM_CLASSES, activation='softmax')
])

In [9]:
model.summary() 

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm (LSTM)                 (None, 30, 128)           133632    
                                                                 
 lstm_1 (LSTM)               (None, 30, 256)           394240    
                                                                 
 lstm_2 (LSTM)               (None, 128)               197120    
                                                                 
 dropout (Dropout)           (None, 128)               0         
                                                                 
 dense (Dense)               (None, 128)               16512     
                                                                 
 dense_1 (Dense)             (None, 64)                8256      
                                                                 
 dense_2 (Dense)             (None, 8)                 5

In [10]:
# компиляция модели
model.compile(optimizer='adam', 
              loss='categorical_crossentropy', 
              metrics=['categorical_accuracy'])

In [11]:
log_dir = os.path.join('mediapipe_pose', 'test')
tb_callback = TensorBoard(log_dir=log_dir)

In [12]:
# обучение модели
model.fit(X_train, y_train, epochs=300, batch_size=128, callbacks=[tb_callback])

Epoch 1/300
13/13 [==============================] - 4s 141ms/step - loss: 1.2973 - categorical_accuracy: 0.6852
Epoch 2/300
13/13 [==============================] - 2s 145ms/step - loss: 1.0420 - categorical_accuracy: 0.7537
Epoch 3/300
13/13 [==============================] - 2s 142ms/step - loss: 0.9495 - categorical_accuracy: 0.7537
Epoch 4/300
13/13 [==============================] - 2s 142ms/step - loss: 0.8914 - categorical_accuracy: 0.7580
Epoch 5/300
13/13 [==============================] - 2s 143ms/step - loss: 0.9366 - categorical_accuracy: 0.7667
Epoch 6/300
13/13 [==============================] - 2s 144ms/step - loss: 0.9106 - categorical_accuracy: 0.7568
Epoch 7/300
13/13 [==============================] - 2s 140ms/step - loss: 0.8302 - categorical_accuracy: 0.7981
Epoch 8/300
13/13 [==============================] - 2s 145ms/step - loss: 0.7941 - categorical_accuracy: 0.8056
Epoch 9/300
13/13 [==============================] - 2s 142ms/step - loss: 0.8841 - categorical_

In [13]:
# Оценка модели на тестовой выборке
tests_res = model.predict(X_test)
accuracy = 0

for i in range(tests_res.shape[0]):
    res_action = actions[np.argmax(tests_res[i])]
    test_action = actions[np.argmax(y_test[i])]
    if(res_action == test_action):
        accuracy += 1
    print(f'Модель: {res_action} | {test_action}')

accuracy /= tests_res.shape[0]
print(f'Точность модели: {accuracy}')

6/6 [==============================] - 0s 14ms/step
Модель: static | static
Модель: static | static
Модель: static | static
Модель: static | down
Модель: static | static
Модель: static | static
Модель: static | static
Модель: static | static
Модель: static | static
Модель: none | none
Модель: static | static
Модель: static | static
Модель: none | none
Модель: static | static
Модель: static | static
Модель: static | static
Модель: static | static
Модель: static | static
Модель: static | static
Модель: static | static
Модель: static | static
Модель: static | static
Модель: static | shake
Модель: static | static
Модель: static | static
Модель: static | static
Модель: static | static
Модель: static | static
Модель: static | to_fist
Модель: static | static
Модель: static | static
Модель: static | static
Модель: static | static
Модель: static | static
Модель: static | static
Модель: static | static
Модель: static | static
Модель: static | static
Модель: static | static
Модель: static | stati

In [14]:
# Model evaluation
val_loss, val_acc = model.evaluate(X_test, y_test, batch_size=128)

2/2 [==============================] - 0s 23ms/step - loss: 0.7707 - categorical_accuracy: 0.8222


In [53]:
# Сохраняем модель
model.save(model_path)

# Проверка работы модели

In [21]:
# Загружаем модель
loaded_model = tf.keras.models.load_model(model_path)

In [24]:
# Model evaluation
val_loss, val_acc = model.evaluate(X_test, y_test, batch_size=128)

2/2 [==============================] - 0s 23ms/step - loss: 680118.2500 - categorical_accuracy: 0.0889


In [25]:
sequence = []

for num_frame in range(sequence_length):
    np_arr = np.load(os.path.join(DATA_PATH, 'down', '45', f"{frame_num}.npy"))

    sequence.append(np_arr)

sequence = np.array(sequence)

# Изменяем форму массива на (1, 126)
np_test = sequence[:30].reshape(1, 30, -1)

print(np_test.shape)

res = np.argmax(loaded_model.predict(np_test))

print(actions[res], res)

(1, 30, 132)
1/1 [==============================] - 0s 239ms/step
down 1


In [30]:
tests_res = loaded_model.predict(X_test)
accuracy = 0

for i in range(tests_res.shape[0]):
    res_action = actions[np.argmax(tests_res[i])]
    test_action = actions[np.argmax(y_test[i])]
    if(res_action == test_action):
        accuracy += 1
    print(f'Модель: {res_action} | {test_action}')

accuracy /= tests_res.shape[0]
print(f'Точность модели: {accuracy}')

3/3 [==============================] - 0s 11ms/step
Модель: to_fist | to_fist
Модель: right | right
Модель: down | down
Модель: z_write | z_write
Модель: right | right
Модель: none | none
Модель: z_write | z_write
Модель: shake | shake
Модель: right | right
Модель: up | up
Модель: down | down
Модель: shake | shake
Модель: to_fist | to_fist
Модель: down | down
Модель: none | none
Модель: to_fist | to_fist
Модель: down | down
Модель: right | right
Модель: static | shake
Модель: shake | shake
Модель: none | none
Модель: down | down
Модель: up | up
Модель: to_fist | to_fist
Модель: down | down
Модель: down | down
Модель: up | up
Модель: none | none
Модель: static | static
Модель: z_write | z_write
Модель: down | down
Модель: static | static
Модель: down | down
Модель: to_fist | to_fist
Модель: shake | static
Модель: static | shake
Модель: up | up
Модель: right | right
Модель: right | static
Модель: z_write | z_write
Модель: none | none
Модель: shake | shake
Модель: to_fist | to_fist
Модель